In [2]:
def csv_FileReader(csvFile):
    import csv
    with open(csvFile, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            print(row)
csvFile = 'accounts.csv'
csv_FileReader(csvFile)

['amohr', '1337', '43.00', '0666', '101.45', '1729', '5000.00', '5000']
['bbaggins', '2890', '15345.49', '2941', '########', '3021', '0.00', '50000']
['emusk', '0001', '21588738', '0002', '########', '0003', '435678.6', '10000000']


In [12]:
class Account():
    '''
    
    '''
    def __init__(self, accountID = '', balance = 0, interest = 0):
        if not isinstance(accountID, str):
            raise TypeError('Account id must be a string')
        
        if len(accountID) != 4:
            raise ValueError('Account id must contain 4 digits')

        for number in accountID:
            if number not in '0123456789':
                raise ValueError('Account id must contain only numbers')
        
        if not isinstance(balance, (int, float)):
            raise TypeError('Balance must be an integer or float')
        
        if not isinstance(interest, (int, float)):
            raise TypeError('Interest must be an integer or float')
        
        if interest < 0:
            raise ValueError('Interest cannot be negative')
        
        self.accountID = accountID
        self.balance = balance
        self.interest = interest
    
    def __str__(self):
        return f'Account ID: {self.accountID} Balance: {self.balance} Interest: {self.interest}%'

class Credit(Account):
    def __init__(self, accountID = '', balance = 0, interest = 0, credit_limit = 0):
        super().__init__(accountID, balance, interest)
        
        if not isinstance(credit_limit, (int, float)):
            raise TypeError('Credit limit must be an int or float')
        
        if credit_limit < balance:
            raise ValueError("Credit limit must be greater than or equal to the balance")

        self.credit_limit = credit_limit

    def __str__(self):
        s = super().__str__()
        return f'{s} Credit limit: {self.credit_limit}'
